In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from xgboost import XGBRegressor
from pathlib import Path

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('BMIProject/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## importing file

In [14]:
print(os.getcwd())
df=pd.read_csv("../data/bmi.csv")
df.head()

d:\WorkSpace\BMIProject\notebooks


FileNotFoundError: [Errno 2] No such file or directory: '../data/bmi.csv'

## Pre-Processing & Data Cleaning

In [ ]:
df.info()
## dataset contain 1 cat variable and 3 num variables, no null data or missing values

: 

In [ ]:
df.describe()
## target variable Index is numeric but it actually depicts categories of BMI

: 

In [ ]:
df['Gender'].value_counts().plot(kind='bar')

#Male and Female datapointa are roughly in equal proportions

: 

## BMI Formula 



**weight (kg) / (height (m) x height (m))**

**weight (kg) / (height (cm) x height (cm)) x 10,000**

***1 cm = 0.393 inch***

***1 kg = 2.204 lb***

**Index :
0 - Extremely Weak,
1 - Weak,
2 - Normal,
3 - Overweight,
4 - Obesity,
5 - Extreme Obesity**

**Classification	BMI range - kg/m2**

**Severe Thinness	< 16     - Extremely Weak**

**Moderate Thinness	16 - 17  - Extremely Weak**

**Mild Thinness	17 - 18.5    - Weak**

**Normal	18.5 - 25        - Normal**

**Overweight	25 - 30      - Overweight**

**Obese 	   30  - 35      - Obesity**

**Obese 	> 35             - Extreme Obesity**

## Visualizations

In [ ]:
cat_columns = df.select_dtypes(include=['object']).columns
numeric_columns = df.select_dtypes(exclude=['object']).columns

##ploting
fig, axs = plt.subplots(1,len(numeric_columns), figsize=(15,7))

for i,column in enumerate(numeric_columns):
    
    axs[i].boxplot(df[column],patch_artist=True)
    axs[i].set_title(column)
    

fig.tight_layout() 
plt.show()

: 

## Transformation

In [ ]:
numeric_columns = numeric_columns.drop('Index')


: 

In [ ]:
numeric_columns, cat_columns

: 

In [ ]:
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

col_transformer = ColumnTransformer(
transformers = [
            ('num_transform',num_transformer,numeric_columns),
            ('cat_transform',cat_transformer,cat_columns)
]
)

: 

In [ ]:
Y = df['Index']
X=df.drop(columns='Index',axis=1)
X.head(),Y.head()

: 

In [ ]:
## transforming X
X = col_transformer.fit_transform(X)


: 

In [ ]:
X.shape

: 

## Splitting Dataset

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=43)

: 

## Training with different models

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor()
    
}

model_results = {
    
}

for name, model in models.items():
    
    model.fit(X_train,Y_train)
    prediction = model.predict(X_test)
    prediction = np.clip(np.round(prediction).astype(int),a_min=0,a_max=5)
    mse = mean_squared_error(Y_test,prediction)
    r2  = r2_score(Y_test,prediction)
    mape = mean_absolute_percentage_error(Y_test,prediction)
    model_results[name]=[mse,r2,mape]
    
    # Create a scatter plot with different colors for true and predicted values
    plt.scatter(Y_test, prediction, c='blue', label='Predicted')
    plt.scatter(Y_test, Y_test, c='red', label='True')
    
       
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title(name)
    plt.legend()
    plt.show()
 
best_model = min(model_results, key=lambda x: model_results[x][0])
print(f'Best model: {best_model}')
print(f'Best model results: {model_results[best_model]}')

: 